In [1]:
%load_ext autoreload
%autoreload 2


In [1]:

from buttermilk import BM
from promptflow.tracing import trace, start_trace
from rich import print as rprint

import os

from hydra import initialize, compose
from omegaconf import OmegaConf

# Load config, specifying overrides for our particular job
with initialize(version_base=None, config_path="./conf"):
    cfg = compose(config_name='config',
                  overrides=["+data=drag", "+step=ordinary", "+save=bq"])

# Load the main ButterMilk singleton instance
# This takes care of credentials, save paths, and other defaults
bm = BM(cfg=cfg)

# Print config
rprint(OmegaConf.to_container(bm.cfg, resolve=True))



/opt/homebrew/Caskroom/miniconda/base/envs/bm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-11-04 17:58:42 J5HW6L4KT6 buttermilk buttermilk.py[ 296] INFO {'message': "Logging setup for: 20241104T0758Z-4K5c-J5HW6L4KT6-suzor. Ready for data collection, saving log to Google Cloud Logs (Resource(type='generic_task', labels={'project_id': 'dmrc-platforms', 'location': 'us-central1', 'namespace': 'automod', 'job': 'judge', 'task_id': '20241104T0758Z-4K5c-J5HW6L4KT6-suzor'})). Default save directory for data in this run is: gs://dmrc-analysis/runs/automod/judge/20241104T0758Z-4K5c-J5HW6L4KT6-suzor", 'project': 'automod', 'job': 'judge', 'run_id': '20241104T0758Z-4K5c-J5HW6L4KT6-suzor', 'save_dir': 'gs://dmrc-analysis/runs/automod/judge/20241104T0758Z-4K5c-J5HW6L4KT6-suzor', 'ip': '159.196.210.27', 'node_name': 'J5HW6L4KT6', 'username': 'suzor'}
2024-11-04 17:58:42 J5HW6L4KT6 buttermilk buttermilk.py[ 304] DEBUG Buttermilk version is: 0.2.0


{
    'name': 'automod',
    'job': 'judge',
    'secret_provider': 'azure',
    'logger': 'gcp',
    'verbose': True,
    'models_secret': 'models',
    'save_dest': 'gcp',
    'save_dir': None,
    'gcp': {'project': 'dmrc-analysis', 'region': 'us-central1', 'bucket': 'dmrc-analysis'},
    'azure': {'vault': 'https://suzorvault.vault.azure.net/', 'resource_group': 'rg-suzor_ai'},
    'aws': {'secret_name': 'secret', 'region': 'us-east-1'},
    'data': [
        {
            'drag': None,
            'name': 'drag queens - alt text',
            'type': 'file',
            'uri': 'gs://dmrc-platforms/data/drag_train.jsonl',
            'columns': {'record_id': 'id', 'content': 'alt_text', 'groundtruth': 'expected'}
        }
    ],
    'step': [
        {
            'name': 'judger',
            'num_runs': 1,
            'concurrent': 20,
            'agent': {
                'name': 'lc_judge',
                'template': 'judge',
                'criteria': 'criteria_ordinary',
                'formatting': 'json_rules',
                'model': ['haiku', 'llama31_70b', 'gpt4o', 'sonnet', 'gemini15pro']
            },
            'parameters': None
        }
    ],
    'run': {'platform': 'local'},
    'save': {
        'type': 'bq',
        'destination': 'dmrc-analysis.toxicity.flow',
        'schema': '/Users/suzor/src/buttermilk/buttermilk/schemas/flow.json'
    }
}

In [2]:
# Create an orchestrator to conduct all combinations of jobs we want to run
from buttermilk.runner.orchestrator import MultiFlowOrchestrator
orchestrator = MultiFlowOrchestrator(step=cfg.step[0], data=cfg.data, save=cfg.save)


I0000 00:00:1730707123.869479 16045583 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


In [3]:
await orchestrator.run_tasks()


2024-11-04 17:59:37 J5HW6L4KT6 buttermilk orchestrator.py[  74] INFO Starting task for Agent judger with job WDQTcYk9RocEJEK9V4jWXT.
2024-11-04 17:59:37 J5HW6L4KT6 buttermilk lc.py[ 185] INFO Invoking chain with haiku...
2024-11-04 17:59:37 J5HW6L4KT6 buttermilk orchestrator.py[  74] INFO Starting task for Agent judger with job dphq7Vub7E4rgAAuxZoHXX.
2024-11-04 17:59:37 J5HW6L4KT6 buttermilk lc.py[ 185] INFO Invoking chain with haiku...
2024-11-04 17:59:37 J5HW6L4KT6 buttermilk orchestrator.py[  74] INFO Starting task for Agent judger with job PBT3rfC3AYuxcLBTD7N6uJ.
2024-11-04 17:59:37 J5HW6L4KT6 buttermilk lc.py[ 185] INFO Invoking chain with haiku...
2024-11-04 17:59:37 J5HW6L4KT6 buttermilk orchestrator.py[  74] INFO Starting task for Agent judger with job hpAHSnShEVMEeSj9hjusQM.
2024-11-04 17:59:37 J5HW6L4KT6 buttermilk lc.py[ 185] INFO Invoking chain with haiku...
2024-11-04 17:59:37 J5HW6L4KT6 buttermilk orchestrator.py[  74] INFO Starting task for Agent judger with job QE738ZE

I0000 00:00:1730707181.686514 16045583 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


2024-11-04 17:59:43 J5HW6L4KT6 buttermilk save.py[ 184] DEBUG Inserting 1 rows to BigQuery table dmrc-analysis.toxicity.flow.
2024-11-04 17:59:43 J5HW6L4KT6 buttermilk save.py[ 191] INFO Successfully pushed 1 rows to BigQuery table dmrc-analysis.toxicity.flow.
2024-11-04 17:59:43 J5HW6L4KT6 buttermilk orchestrator.py[  80] INFO Agent judger completed job WDQTcYk9RocEJEK9V4jWXT successfully.
2024-11-04 17:59:43 J5HW6L4KT6 buttermilk lc.py[ 196] INFO Invoked chain with haiku in 5.84 seconds
2024-11-04 17:59:45 J5HW6L4KT6 buttermilk save.py[ 184] DEBUG Inserting 1 rows to BigQuery table dmrc-analysis.toxicity.flow.
2024-11-04 17:59:46 J5HW6L4KT6 buttermilk save.py[ 191] INFO Successfully pushed 1 rows to BigQuery table dmrc-analysis.toxicity.flow.
2024-11-04 17:59:46 J5HW6L4KT6 buttermilk orchestrator.py[  80] INFO Agent judger completed job hpAHSnShEVMEeSj9hjusQM successfully.
2024-11-04 17:59:46 J5HW6L4KT6 buttermilk lc.py[ 196] INFO Invoked chain with haiku in 8.36 seconds


I0000 00:00:1730707186.233406 16045583 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


2024-11-04 17:59:47 J5HW6L4KT6 buttermilk save.py[ 184] DEBUG Inserting 1 rows to BigQuery table dmrc-analysis.toxicity.flow.
2024-11-04 17:59:48 J5HW6L4KT6 buttermilk save.py[ 191] INFO Successfully pushed 1 rows to BigQuery table dmrc-analysis.toxicity.flow.
2024-11-04 17:59:48 J5HW6L4KT6 buttermilk orchestrator.py[  80] INFO Agent judger completed job PBT3rfC3AYuxcLBTD7N6uJ successfully.
2024-11-04 17:59:48 J5HW6L4KT6 buttermilk lc.py[ 185] INFO Invoking chain with haiku...
2024-11-04 17:59:48 J5HW6L4KT6 buttermilk orchestrator.py[  74] INFO Starting task for Agent judger with job e5LMkfvUqc3nEFBEgfR9BQ.
2024-11-04 17:59:48 J5HW6L4KT6 buttermilk lc.py[ 185] INFO Invoking chain with llama31_70b...
2024-11-04 17:59:48 J5HW6L4KT6 buttermilk lc.py[ 185] INFO Invoking chain with haiku...
2024-11-04 17:59:48 J5HW6L4KT6 buttermilk orchestrator.py[  74] INFO Starting task for Agent judger with job 87gNzc7naLzMbcbGoa6PFT.
2024-11-04 17:59:48 J5HW6L4KT6 buttermilk lc.py[ 185] INFO Invoking ch

...

2024-11-04 18:03:18 J5HW6L4KT6 buttermilk save.py[ 184] DEBUG Inserting 1 rows to BigQuery table dmrc-analysis.toxicity.flow.
2024-11-04 18:03:19 J5HW6L4KT6 buttermilk save.py[ 191] INFO Successfully pushed 1 rows to BigQuery table dmrc-analysis.toxicity.flow.
2024-11-04 18:03:19 J5HW6L4KT6 buttermilk orchestrator.py[  80] INFO Agent judger completed job LbHBtrpAsS4zH5TfJDSVxG successfully.
2024-11-04 18:03:19 J5HW6L4KT6 buttermilk lc.py[ 196] INFO Invoked chain with gemini15pro in 4.15 seconds


I0000 00:00:1730707399.210195 16045583 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1730707399.790190 16045583 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


2024-11-04 18:03:20 J5HW6L4KT6 buttermilk save.py[ 184] DEBUG Inserting 1 rows to BigQuery table dmrc-analysis.toxicity.flow.
2024-11-04 18:03:21 J5HW6L4KT6 buttermilk save.py[ 191] INFO Successfully pushed 1 rows to BigQuery table dmrc-analysis.toxicity.flow.
2024-11-04 18:03:21 J5HW6L4KT6 buttermilk orchestrator.py[  80] INFO Agent judger completed job LMJJEqsEBtGAQnbQTAHhLN successfully.
2024-11-04 18:03:21 J5HW6L4KT6 buttermilk lc.py[ 196] INFO Invoked chain with gemini15pro in 6.12 seconds


I0000 00:00:1730707401.179912 16045583 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1730707401.906133 16045583 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


2024-11-04 18:03:23 J5HW6L4KT6 buttermilk save.py[ 184] DEBUG Inserting 1 rows to BigQuery table dmrc-analysis.toxicity.flow.
2024-11-04 18:03:23 J5HW6L4KT6 buttermilk save.py[ 191] INFO Successfully pushed 1 rows to BigQuery table dmrc-analysis.toxicity.flow.
2024-11-04 18:03:23 J5HW6L4KT6 buttermilk orchestrator.py[  80] INFO Agent judger completed job jd2wVgAMzovdJ7ccAu7iSB successfully.
2024-11-04 18:03:23 J5HW6L4KT6 buttermilk orchestrator.py[  94] INFO Completed run 1 of 1
2024-11-04 18:03:23 J5HW6L4KT6 buttermilk orchestrator.py[  98] INFO All tasks have completed.
